# Berechnung Lampen

## 1. Importieren der Bibliotheken

In [2]:
import math
import csv
from datetime import datetime, timedelta
from collections import defaultdict

## 2. Festlegen der Eingangswerte

In [3]:
# Leistungsaufnahme einer Lampe in Watt für 1,44 m^2 (120cm * 120cm)
leistungsaufnahme_einzeln = 632                                        # W
abdeckungsflaeche_einzeln = 2.5                                        # m²

# Größe des Gewächshauses
flaeche = 10000                                                        # m² (1 Hektar)

# Schwellwert für ausreichende Solareinstrahlung
SOLAR_THRESHOLD = 100                                                  # W/m²

# Lichtzeit von 6:00 - 20:00 Uhr (14 Stunden)
LIGHT_START_HOUR = 6
LIGHT_END_HOUR = 20

## 3. Berechnungen

In [4]:
# Anzahl der Lampen, aufgerundet auf die nächste ganze Zahl
anzahl_lampen = math.ceil(flaeche / abdeckungsflaeche_einzeln)

# Energieverbrauch gesamtes Gewächshaus pro Stunde in Watt
energieverbrauch_gesamt_stunde = anzahl_lampen * leistungsaufnahme_einzeln

# Solareinstrahlung einlesen aus bereinigter CSV (Kombination aus Bochum und Bremen)
solar_data = {}
with open('Solareinstrahlung_Bochum_Bremen.csv', 'r', encoding='utf-8') as f:
    # Erste Zeile überspringen (Header)
    next(f)
    for line in f:
        parts = line.strip().split(';')
        if len(parts) < 2:
            continue
        
        # Datum im Format YYYYMMDDHH
        datum_str = parts[0]
        try:
            timestamp = datetime.strptime(datum_str, '%Y%m%d%H')
            
            # Solar_W_m2 Wert (bereits in W/m² umgerechnet)
            solar_w_m2 = float(parts[1].replace(',', '.'))
            
            solar_data[timestamp] = solar_w_m2
        except (ValueError, IndexError):
            continue
# Stündliche Energieverbrauchsdaten berechnen
results = []

# Für jede Stunde des Jahres 2019
start_date = datetime(2019, 1, 1, 0, 0, 0)
end_date = datetime(2020, 1, 1, 0, 0, 0)

current_time = start_date

while current_time < end_date:
    # Stunde des Tages (0-23)
    hour_of_day = current_time.hour
    
    # Prüfen, ob wir im Lichtzeitfenster sind (6:00 - 20:00 Uhr)
    if LIGHT_START_HOUR <= hour_of_day < LIGHT_END_HOUR:
        # Im Lichtzeitfenster: Prüfe Solareinstrahlung
        solar_radiation = solar_data.get(current_time, 0)
        
        if solar_radiation < SOLAR_THRESHOLD:
            # Nicht genug Sonnenlicht > Lampen an
            energy_kw = energieverbrauch_gesamt_stunde / 1000
        else:
            # Genug Sonnenlicht > Lampen aus
            energy_kw = 0
    else:
        # Außerhalb des Lichtzeitfensters > Lampen aus
        energy_kw = 0
    
    # Datum/Uhrzeit-Format: YYYYMMDDHH
    timestamp = current_time.strftime('%Y%m%d%H')
    results.append([timestamp, round(energy_kw, 2)])
    
    # Zur nächsten Stunde
    current_time += timedelta(hours=1)

# Statistik berechnen
total_lamp_hours = sum(1 for r in results if r[1] > 0)
total_energy_kwh = sum(r[1] for r in results)  # bereits in kW, Summe ergibt kWh

## 4. Augabe von Kontollwerten

In [5]:
#Lampen Anzahl ausgeben
print(f"Anzahl Lampen: {anzahl_lampen}")
#Energieverbrauch ausgeben
print(f"Energieverbrauch pro Stunde (wenn alle an): {energieverbrauch_gesamt_stunde} W")
#Länge der Stundenwerte ausgeben
print(f"Solareinstrahlung-Daten geladen: {len(solar_data)} Stunden")

#Werte der Berechnung ausgeben
print(f"\nStatistik für 2019:")
print(f"Gesamte Stunden mit Lampenbetrieb: {total_lamp_hours}")
print(f"Gesamter Energieverbrauch: {total_energy_kwh:.2f} kWh")

Anzahl Lampen: 4000
Energieverbrauch pro Stunde (wenn alle an): 2528000 W
Solareinstrahlung-Daten geladen: 147527 Stunden

Statistik für 2019:
Gesamte Stunden mit Lampenbetrieb: 2109
Gesamter Energieverbrauch: 5331552.00 kWh


## 5. Daten als CSV ausgeben

In [6]:
# Ergebnisse in CSV-Datei schreiben
output_file = 'hourly_lamp_energy_2019.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['DateTime', 'Energy_kW'])
    writer.writerows(results)

print(f"\nErgebnisse gespeichert in: {output_file}")
print(f"Anzahl Datensätze: {len(results)}")


Ergebnisse gespeichert in: hourly_lamp_energy_2019.csv
Anzahl Datensätze: 8760
